# Библиотеки

In [ ]:
import numpy as np
import pandas as pd

import scipy
from scipy import stats
import statsmodels.formula.api as smf

import matplotlib.pyplot as plt
import seaborn as sns

import copy

import tqdm
import tqdm.notebook as tqdm

import sklearn
from sklearn.ensemble import RandomForestRegressor

plt.style.use('ggplot')

# Стратификация

## Данные

### Задаем ошибки 1 и 2 рода, ожидаемый эффект. Вычисляем необходимое число наблюдений в каждой группе

![](img/effect_size.png)

In [ ]:
def get_sample_size(alpha, beta, std_a, std_b, effect):
    '''
    Вычисляет количество наблюдений в каждой группе при заданных:
    alpha, beta - ошибках 1 и 2 рода, 
    std_a, std_b - стандартных отколениях в контрольной и экспериментальной группах,
    effect - минимальном ожидаемом эффекте
    '''
    norm_rv = stats.norm(loc=0, scale=1)
    t_alpha = norm_rv.ppf(1 - alpha / 2)
    t_beta = norm_rv.ppf(1 - beta)
    var = (std_a**2) + (std_b)**2
    sample_size = int((t_alpha + t_beta) ** 2 * var / (effect ** 2))
    return sample_size

In [ ]:
alpha = 0.05
beta = 0.2
std_a = 800
std_b = 800
effect = 100

sample_size = get_sample_size(alpha, beta, std_a, std_b, effect)
sample_size

### Проверяем расчеты: ошибку первого рода на АА и ошибку второго рода на АВ

In [ ]:
mu_control = 2500
mu_pilot = mu_control + effect
std = 800

first_type_errors = []
second_type_errors = []

for i in tqdm.tqdm(range(10000)):
    
    control_one = stats.norm(mu_control, std).rvs(sample_size)
    control_two = stats.norm(mu_control, std).rvs(sample_size)
    pilot = stats.norm(mu_pilot, std).rvs(sample_size)
    
    _, pvalue_aa = stats.ttest_ind(control_one, control_two)
    first_type_errors.append(pvalue_aa < alpha)
    _, pvalue_ab = stats.ttest_ind(control_one, pilot)
    second_type_errors.append(pvalue_ab >= alpha)

prop_first_type_errors = np.mean(first_type_errors)
prop_second_type_errors = np.mean(second_type_errors)
print(f'prop_first_type_errors = {prop_first_type_errors:0.3f}')
print(f'prop_second_type_errors = {prop_second_type_errors:0.3f}')

### Фунцкии для случайного и стратифицированного семплирования, а также сам тест

In [ ]:
def get_random_data(strats, sample_size, strat_to_param, effect = 0):
    """
    Генерирует данные случайным семплированием.
    Возвращает датафрейм со значениями метрики и номерами страт пользователей
    в контрольной и экспериментальной группах:
    strats - cписок с распределением страт в популяции,
    sample_size - размеры групп,
    strat_to_param - словарь с параметрами страт,
    effect - размер эффекта
    """
    
    control_strats, pilot_strats = np.random.choice(strats, (2, sample_size), False)
    
    control, pilot = [], []
    for strat, (_, mu, std) in strat_to_param.items():
        n_control = np.sum(control_strats == strat)
        n_pilot = np.sum(pilot_strats == strat)
        control += [(x, strat) for x in stats.norm(mu, std).rvs(n_control)]
        pilot += [(x, strat) for x in stats.norm(mu + effect, std).rvs(n_pilot)]
        
    control_df = pd.DataFrame(control, columns = ['value', 'strat'])
    pilot_df = pd.DataFrame(pilot, columns = ['value', 'strat'])
    return control_df, pilot_df

def get_stratified_data(strat_to_param, effect = 0):
    """
    Генерирует данные стратифицированным семплированием.
    Возвращает датафрейм со значениями метрики и номерами страт пользователей
    в контрольной и экспериментальной группах:
    strat_to_param - словарь с параметрами страт
    effect - размер эффекта
    """
    control, pilot = [], []
    for strat, (n, mu, std) in strat_to_param.items():
        control += [(x, strat) for x in stats.norm(mu, std).rvs(n)]
        pilot += [(x, strat) for x in stats.norm(mu + effect, std).rvs(n)]

    control_df = pd.DataFrame(control, columns = ['value', 'strat'])
    pilot_df = pd.DataFrame(pilot, columns = ['value', 'strat'])
    return control_df, pilot_df

def ttest(a: pd.DataFrame, b: pd.DataFrame):
    """
    Возвращает p-value для теста Стьюдента (при равных дисперсиях):
    a, b - датафреймы пользователей контрольной и экспериментальной групп
    """
    _, pvalue = stats.ttest_ind(a['value'].values, b['value'].values)
    return pvalue

## До коррекции:

### Зная, что эффекта нет (effect = 0, АА тест), вычисляем ошибку первого рода при случайном и стратифицированном семплировании

In [ ]:
N = 10000                           # общее количество пользователей в популяции
w_one, w_two = 0.5, 0.5             # доли страт в популяции
N_one = int(N * w_one)              # количество пользователей первой страты
N_two = int(N * w_two)              # количество пользователей второй страты
mu_one, mu_two = 2000, 3000         # средние значения метрики в стратах
std_one, std_two = 625, 625         # стандартное отклонение метрики в стратах

# ! Будем считать, что при объединении страт в контрольной и экспериментальной группах стандартное отклоение составит 800
# (именно для такого стандартного отклонения мы вначале рассчитывали количество наблюдений в группах)
    
strats = [1 for _ in range(N_one)] + [2 for _ in range(N_two)]

sample_size = get_sample_size(alpha, beta, std_a, std_b, effect) + 100 # возьмем количество наблюдений в группах с запасом 
                                                                       # на 100 больше, чем нужно
sample_size_one = int(sample_size * w_one)
sample_size_two = int(sample_size * w_two)

strat_to_param = {1: (sample_size_one, mu_one, std_one), 2: (sample_size_two, mu_two, std_two)}

random_first_type_errors = []
stratified_first_type_errors = []
random_deltas = []
stratified_deltas = []

for _ in tqdm.tqdm(range(10000)):
    # по умолчанию в функции генерации выборок эффекта нет
    control_random, pilot_random = get_random_data(strats, sample_size, strat_to_param)
    control_stratified, pilot_stratified = get_stratified_data(strat_to_param)
    
    random_deltas.append(pilot_random['value'].mean() - control_random['value'].mean())
    stratified_deltas.append(pilot_stratified['value'].mean() - control_stratified['value'].mean())

    pvalue_random = ttest(control_random, pilot_random)
    random_first_type_errors.append(pvalue_random < alpha)
    
    pvalue_stratified = ttest(control_stratified, pilot_stratified)
    stratified_first_type_errors.append(pvalue_stratified < alpha)

prop_random_first_type_errors = np.mean(random_first_type_errors)
prop_stratified_first_type_errors = np.mean(stratified_first_type_errors)
print(f'prop_random_first_type_errors = {prop_random_first_type_errors:0.3f}')
print(f'prop_stratified_first_type_errors = {prop_stratified_first_type_errors:0.3f}')

### Зная, что эффект есть (effect = 100, АВ тест), вычисляем ошибку второго рода при случайном и стратифицированном семплировании

In [ ]:
effect = 100
random_second_type_errors = []
stratified_second_type_errors = []

for _ in tqdm.tqdm(range(10000)):
    # добавляем эффект в функцию генерации
    control_random, pilot_random = get_random_data(strats, sample_size, strat_to_param, effect)
    control_stratified, pilot_stratified = get_stratified_data(strat_to_param, effect)
    
    pvalue_random = ttest(control_random, pilot_random)
    random_second_type_errors.append(pvalue_random >= alpha)
                                     
    pvalue_stratified = ttest(control_stratified, pilot_stratified)
    stratified_second_type_errors.append(pvalue_stratified >= alpha)

prop_random_second_type_errors = np.mean(random_second_type_errors)
prop_stratified_second_type_errors = np.mean(stratified_second_type_errors)
print(f'prop_random_second_type_errors = {prop_random_second_type_errors:0.3f}')
print(f'prop_stratified_second_type_errors = {prop_stratified_second_type_errors:0.3f}')

Можно заметить, что при случайном семплировании ошибки первого и второго рода остались на прежнем уровне (впрочем, как и должно быть), а при стратифицированном семплировании ошибки первого и второго рода снизилась 

In [ ]:
sns.displot({'Случайное семплирвоание': random_deltas, 'Стратифицированное семплирование': stratified_deltas}, kind='kde')
plt.title('Распределение разницы средних при разных видах семплирования')
plt.ylabel('Плотность')

random_var_deltas = np.array(random_deltas).var(ddof = 1)
stratified_var_deltas = np.array(stratified_deltas).var(ddof = 1)

print(f'Дисперсия разницы средних при случаном семплировании: {random_var_deltas}')
print(f'Дисперсия разницы средних при стратифицированном семплировании: {stratified_var_deltas}')

print(f'Произошло снижение дисперсии разницы средних в {round(random_var_deltas / stratified_var_deltas, 2)} раз(а)')

## После коррекции:

Исправляем тест Стьюдента: вносим корректировку на стратифицированное семплирование (стратифицированный подсчет среднего)

In [ ]:
def calc_strat_mean(df: pd.DataFrame, weights: pd.Series):
    """
    Считает стратифицированное среднее:
    df - датафрейм с целевой метрикой и стратами, к которой относится пользователь
    weights - серия {название страты: вес страты в популяции}
    """
    strat_mean = df.groupby('strat')['value'].mean()
    return (strat_mean * weights).sum()

def calc_strat_var(df: pd.DataFrame, weights: pd.Series):
    """
    Считает стратифицированную дисперсию:
    df - датафрейм с целевой метрикой и стратами, к которой относится пользователь
    weights - серия {название страты: вес страты в популяции}
    """
    strat_var = df.groupby('strat')['value'].var(ddof = 1)
    return (strat_var * weights).sum()

def ttest_strat(a: pd.DataFrame, b: pd.DataFrame, weights: pd.Series, return_deltas = True):
    """
    Возвращает pvalue теста Стьюдента (при равных дисперсиях) при подсчете стратифицированного среднего:
    a, b - данные пользователей контрольной и экспериментальной групп
    weights - серия {название страты: вес страты в популяции}
    """
    a_strat_mean = calc_strat_mean(a, weights)
    b_strat_mean = calc_strat_mean(b, weights)
    a_strat_var = calc_strat_var(a, weights)
    b_strat_var = calc_strat_var(b, weights)
    delta = a_strat_mean - b_strat_mean
    std = (a_strat_var / len(a) + b_strat_var / len(b)) ** 0.5
    t = delta / std
    
    pvalue = 2 * (1 - stats.t(len(a) + len(b)).cdf(np.abs(t)))
    
#     Для случая неравных дисперсий:
    
#     v = ((a_strat_var / len(a) + b_strat_var / len(b)) ** 2) / \
#         ((a_strat_var ** 2 / (len(a) ** 2 * (len(a) - 1))) + \
#         (b_strat_var ** 2 / (len(b) ** 2 * (len(b) - 1)))) # приближение Уэлча
#     pvalue = 2 * (1 - stats.t(v).cdf(np.abs(t)))

    if return_deltas == True:
        return (pvalue, delta)
    else:
        return pvalue

### Вычисляем ошибку первого (на АА тесте) и второго (на АВ тесте) рода при стратифицированном семплировании и стратифицированном среднем (и соответственно стратифицированной дисперсии)

In [ ]:
weights = pd.Series({1: w_one, 2: w_two})

first_type_errors = []
second_type_errors = []

stratified_deltas_strat_mean = []

for _ in tqdm.tqdm(range(10000)):
    control_aa, pilot_aa = get_stratified_data(strat_to_param)
    control_ab, pilot_ab = get_stratified_data(strat_to_param, effect)

    pvalue_aa, delta = ttest_strat(control_aa, pilot_aa, weights, return_deltas = True)
    first_type_errors.append(pvalue_aa < alpha)
    stratified_deltas_strat_mean.append(delta)
    
    pvalue_ab = ttest_strat(control_ab, pilot_ab, weights, return_deltas = False)
    second_type_errors.append(pvalue_ab >= alpha)
    
prop_first_type_errors = np.mean(first_type_errors)
prop_second_type_errors = np.mean(second_type_errors)
print(f'prop_first_type_errors = {prop_first_type_errors:0.3f}')
print(f'prop_second_type_errors = {prop_second_type_errors:0.3f}')
print(f'Произошло снижение ошибки II рода в {round(prop_random_second_type_errors / prop_second_type_errors, 2)} раз(а)')

Ошибка первого рода осталась на запланированном уровне, а ошибка второго рода снизилась (0.2 -> 0.035), а значит мощность теста повысилась

In [ ]:
sns.displot({'Случайное семплирвоание': random_deltas, 'Стратифицированное семплирование \n (и стратифицированное среднее)': stratified_deltas_strat_mean}, kind='kde')
plt.title('Распределение разницы средних при разных видах семплирования')
plt.ylabel('Плотность')

random_var_deltas = np.array(random_deltas).var(ddof = 1)
stratified_var_deltas = np.array(stratified_deltas_strat_mean).var(ddof = 1)

print(f'Дисперсия разницы средних при случаном семплировании: {random_var_deltas}')
print(f'Дисперсия разницы средних при стратифицированном семплировании (и стратифицированном среднем): {stratified_var_deltas}')

print(f'Произошло снижение дисперсии разницы средних в {round(random_var_deltas / stratified_var_deltas, 2)} раз(а)')

### Вычисляем ошибку первого (на АА тесте) и второго (на АВ тесте) рода при случайном семплировании и стратифицированном среднем (и соответственно стратифицированной дисперсии) (постстратификация)

In [ ]:
first_type_errors = []
second_type_errors = []

for _ in tqdm.tqdm(range(10000)):
    # Случайное семплирование!
    control_aa, pilot_aa = get_random_data(strats, sample_size, strat_to_param)
    control_ab, pilot_ab = get_random_data(strats, sample_size, strat_to_param, effect)
    
    # Стратифицированный подсчет среднего и дисперсии!
    pvalue_aa, delta = ttest_strat(control_aa, pilot_aa, weights, return_deltas = True)
    first_type_errors.append(pvalue_aa < alpha)
    stratified_deltas_strat_mean.append(delta)
    
    pvalue_ab = ttest_strat(control_ab, pilot_ab, weights, return_deltas = False)
    second_type_errors.append(pvalue_ab >= alpha)

prop_first_type_errors = np.mean(first_type_errors)
prop_second_type_errors = np.mean(second_type_errors)
print(f'prop_first_type_errors = {prop_first_type_errors:0.3f}')
print(f'prop_second_type_errors = {prop_second_type_errors:0.3f}')
print(f'Произошло снижение ошибки II рода в {round(prop_random_second_type_errors / prop_second_type_errors, 2)} раз(а)')

In [ ]:
sns.displot({'Случайное семплирвоание': random_deltas, 'Cлучайное семплирование \n (и стратифицированное среднее)': stratified_deltas_strat_mean}, kind='kde')
plt.title('Распределение разницы средних при разных видах усреднения')
plt.ylabel('Плотность')

random_var_deltas = np.array(random_deltas).var(ddof = 1)
stratified_var_deltas = np.array(stratified_deltas_strat_mean).var(ddof = 1)

print(f'Дисперсия разницы средних при случаном семплировании: {random_var_deltas}')
print(f'Дисперсия разницы средних при случаном семплировании (и стратифицированном среднем): {stratified_var_deltas}')

print(f'Произошло снижение дисперсии разницы средних в {round(random_var_deltas / stratified_var_deltas, 2)} раз(а)')

При достаточно большом объёме данных отличия между стратификацией и постстратификацией минимальны

# Utilizing pre-experiment data (UED)

![](img/corr_theorem.png)

In [ ]:
def generate_correlated_variables(rho, mu_x, std_x, mu_y, std_y, n):
    '''
    Генерирует две случайные величины с заранее заданной корреляцией.
    Предварителньо задаются параментры величин:
    mu_x, std_x - математическое ожидание и стандартное отклонение СВ X
    mu_y, std_y - математическое ожидание и стандартное отклонение СВ Y
    rho - коэффициент корреляции Пирсона между СВ Х и СВ Y
    n - число наблюдений
    '''
    
    Z_1 = stats.norm().rvs(n)
    Z_2 = stats.norm().rvs(n)
    
    X = std_x * Z_1 + mu_x

    Y = std_y * (rho * Z_1 + (1 - rho ** 2)**0.5 * Z_2) + mu_y

    return X, Y

In [ ]:
X, Y = generate_correlated_variables(0.8, 4, 2, 4, 2, 1000)
scipy.stats.pearsonr(X, Y)[0]

## Данные

In [ ]:
n = 1100*2

treatment_effect = 100

mu_pre = 2500
std_pre = 800

mu_exp = 2500
std_exp = 800

corr = 0.7

metric_pre, metric_exp = generate_correlated_variables(corr, mu_pre, std_pre, mu_exp, std_exp, n)

is_treatment = np.concatenate((np.zeros(n//2, dtype = int), np.ones(n//2, dtype = int)), axis=None)
np.random.shuffle(is_treatment)

data = pd.DataFrame({'metric_pre': metric_pre,
                     'metric_exp': metric_exp,
                     'is_treatment': is_treatment})

data.loc[data.is_treatment == 1, 'metric_exp'] += treatment_effect

data.head()

In [ ]:
print(f'Корреляция между метрикой до эксперимента\nи во время эксперимента: {scipy.stats.pearsonr(data["metric_pre"], data["metric_exp"])[0]}')
print()
print('Среднее значение метрики:')
print(f"До эксперимента в контрольной группе: {round(data[data.is_treatment == 0]['metric_pre'].mean(), 2)}")
print(f"До эксперимента в тестовой группе: {round(data[data.is_treatment == 1]['metric_pre'].mean(), 2)}")
print(f"Во время эксперимента в контрольной группе: {round(data[data.is_treatment == 0]['metric_exp'].mean(), 2)}")
print(f"Во время эксперимента в тестовой группе: {round(data[data.is_treatment == 1]['metric_exp'].mean(), 2)}")
print(f"До эксперимента: {round(data['metric_pre'].mean(), 2)}")
print(f"Во время эксперимента: {round(data['metric_exp'].mean(), 2)}")
print()
print('Стандартное отклонение метрики:')
print(f"До эксперимента в контрольной группе: {round(data[data.is_treatment == 0]['metric_pre'].std(ddof = 1), 2)}")
print(f"До эксперимента в тестовой группе: {round(data[data.is_treatment == 1]['metric_pre'].std(ddof = 1), 2)}")
print(f"Во время эксперимента в контрольной группе: {round(data[data.is_treatment == 0]['metric_exp'].std(ddof = 1), 2)}")
print(f"Во время эксперимента в тестовой группе: {round(data[data.is_treatment == 1]['metric_exp'].std(ddof = 1), 2)}")
print(f"До эксперимента: {round(data['metric_pre'].std(ddof = 1), 2)}")
print(f"Во время эксперимента: {round(data['metric_exp'].std(ddof = 1), 2)}")

Проведем АВ-тесты в разных вариантах:

## Парная регрессия

In [ ]:
model = smf.ols('metric_exp ~ is_treatment', data = data).fit()
model.summary().tables[1]

In [ ]:
_, pvalue = stats.ttest_ind(data[data['is_treatment'] == 1].metric_exp, data[data['is_treatment'] == 0].metric_exp)
print(f'{pvalue:0.10f}')

## Добавляем в регрессию ковариант

In [ ]:
model = smf.ols('metric_exp ~ metric_pre + is_treatment', data = data).fit()
model.summary().tables[1]

In [ ]:
print(f'{model.pvalues.is_treatment:0.10f}')

# CUPED

$$Y_{CUPED} = Y - \theta X$$

$$\theta = \frac{cov(X, Y)}{Var(X)}$$

## Считаем $\theta$

Так:

In [ ]:
cov_X_Y = np.cov(data['metric_pre'], 
                 data['metric_exp'],
                 ddof=1)[0, 1]

var_X = np.var(data['metric_pre'], ddof=1)

theta = cov_X_Y / var_X

theta

Или так:

In [ ]:
model = smf.ols('metric_exp ~ metric_pre', data = data).fit()
model.summary().tables[1]

In [ ]:
theta = model.params[1]

## I способ подсчета $Y_{CUPED}$

In [ ]:
data['metric_exp_cuped_1'] = data['metric_exp'] - (theta * (data['metric_pre'] - data['metric_pre'].mean()))
data.head()

In [ ]:
std_metr_exp =  data['metric_exp'].std(ddof = 1)
std_metr_exp_cuped_1 =  data['metric_exp_cuped_1'].std(ddof = 1)

print(f'Стандартное отклонение метрики \nво время эксперимента: {round(std_metr_exp, 2)}')
print(f'Стандартное отклонение метрики \nво время эксперимента \nпосле cuped-преобразования: {round(std_metr_exp_cuped_1, 2)}')
print(f'После преобразования стандартное \nотклонение метрики во время \nэксперимента уменьшилась на {round(100 - std_metr_exp_cuped_1 * 100 / std_metr_exp, 2)}%')

Проверка формул: 
* падение дисперсии среднего значения метрики в $[1 - {\rho}^2(X, Y)]$ раз 
* неизменность математического ожидания метрики

In [ ]:
var_Y = np.std(data['metric_exp'])**2
var_Y_mean = var_Y / len(data['metric_exp'])

var_Y_cuped_1 = np.std(data['metric_exp_cuped_1'])**2
var_Y_cuped_1_mean = var_Y_cuped_1 / len(data['metric_exp_cuped_1'])

corr_X_Y = np.corrcoef([data['metric_pre'], 
                        data['metric_exp']])[0,1]

print(f'Дисперсия среднего\n исходной метрики = {var_Y_mean:0.2f}')
print(f'Дисперсия среднего\n cuped-преобразованной метрики = {var_Y_cuped_1_mean:0.2f}')
print(f'(1 - rho^2) = {1 - corr_X_Y**2:0.3f}')
print(f'Отношение дисперсий = {var_Y_cuped_1_mean / var_Y_mean:0.3f}')

print()

print(f'Cреднее значение метрики\n до преобразования: {round(data["metric_exp"].mean(), 2)}')
print(f'Cреднее значение метрики\n после преобразования: {round(data["metric_exp_cuped_1"].mean(), 2)}')

In [ ]:
model = smf.ols('metric_exp_cuped_1 ~ is_treatment', data = data).fit()
model.summary().tables[1]

In [ ]:
_, pvalue = stats.ttest_ind(data[data['is_treatment'] == 1].metric_exp_cuped_1, 
                            data[data['is_treatment'] == 0].metric_exp_cuped_1)
print(f'{pvalue:0.10f}')

## II способ подсчета $Y_{CUPED}$

In [ ]:
model = smf.ols('metric_exp ~ metric_pre', data = data).fit()

data['metric_exp_cuped_2'] = model.resid + data['metric_exp'].mean()
data.head()

In [ ]:
std_metr_exp =  data['metric_exp'].std(ddof = 1)
std_metr_exp_cuped_2 =  data['metric_exp_cuped_2'].std(ddof = 1)

print(f'Стандартное отклонение метрики во время эксперимента: {round(std_metr_exp, 2)}')
print(f'Стандартное отклонение метрики во время эксперимента после cuped-преобразования: {round(std_metr_exp_cuped_2, 2)}')
print(f'После преобразования стандартное отклонение метрики во время эксперимента уменьшилась на {round(100 - std_metr_exp_cuped_2 * 100 / std_metr_exp, 2)}%')

Проверка формул: 
* падение дисперсии среднего значения метрики в $[1 - {\rho}^2(X, Y)]$ раз 
* неизменность математического ожидания метрики

In [ ]:
var_Y = np.std(data['metric_exp'])**2
var_Y_mean = var_Y / len(data['metric_exp'])

var_Y_cuped_2 = np.std(data['metric_exp_cuped_2'])**2
var_Y_cuped_2_mean = var_Y_cuped_2 / len(data['metric_exp_cuped_2'])

corr_X_Y = np.corrcoef([data['metric_pre'], 
                        data['metric_exp']])[0,1]

print(f'Var(Y_mean) = {var_Y_mean:0.2f}')
print(f'Var(Y_cuped_2_mean) = {var_Y_cuped_2_mean:0.2f}')
print(f'(1 - rho^2) = {1 - corr_X_Y**2:0.3f}')
print(f'Var(Y_cuped_2_mean) / Var(Y_mean) = {var_Y_cuped_2_mean / var_Y_mean:0.3f}')

print()

print(f'Cреднее значение метрики до преобразования: {data["metric_exp"].mean()}')
print(f'Cреднее значение метрики после преобразования: {data["metric_exp_cuped_2"].mean()}')

In [ ]:
model = smf.ols('metric_exp_cuped_2 ~ is_treatment', data = data).fit()
model.summary().tables[1]

In [ ]:
_, pvalue = stats.ttest_ind(data[data['is_treatment'] == 1].metric_exp_cuped_2,  
                            data[data['is_treatment'] == 0].metric_exp_cuped_2)
print(f'{pvalue:0.10f}')

## Визуализация

Наглядное сокращение диспресии преобразованной метрики (хвосты распределения становятся менее "тяжелыми"):

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

ax_1 = axs[0]
ax_2 = axs[1]

sns.kdeplot(x = data[data.is_treatment == 0]['metric_exp'], 
            data = data, label = 'Initial', fill = True, alpha = 0.1, color = 'red', ax = ax_1)
sns.kdeplot(x = data[data.is_treatment == 0]['metric_exp_cuped_3'], 
            data = data, label = 'CUPED transformed', fill = True, alpha = 0.1, color = 'blue', ax = ax_1)
ax_1.set_title('Without treatment')
ax_1.legend()

sns.kdeplot(x = data[data.is_treatment == 1]['metric_exp'],
            data = data, label = 'Initial', fill = True, alpha = 0.1, color = 'red', ax = ax_2)
sns.kdeplot(x = data[data.is_treatment == 1]['metric_exp_cuped_3'], 
            data = data, label = 'CUPED transformed', fill = True, alpha = 0.1, color = 'blue', ax = ax_2)
ax_2.set_title('With treatment')
ax_2.legend()

fig, axs = plt.subplots(1, 1, figsize=(16, 6))
sns.kdeplot(x = 'metric_exp', 
            data = data, label = 'Initial', fill = True, alpha = 0.1, color = 'red', ax = axs)
sns.kdeplot(x = 'metric_exp_cuped_3', 
            data = data, label = 'CUPED transformed', fill = True, alpha = 0.1, color = 'blue', ax = axs)
axs.set_title('All users')
axs.legend()
plt.show();

## Ошибки I и II рода

In [ ]:
n = 1100*2

alpha = 0.05

treatment_effect = 100

mu_pre = 2500
std_pre = 800

mu_exp = 2500
std_exp = 800

corr = 0.8

simple_first_type_errors = []
simple_second_type_errors = []

cuped_first_type_errors = []
cuped_second_type_errors = []

simple_deltas = []
cuped_deltas = []

for _ in tqdm.tqdm(list(range(10000))):
    
    metric_pre, metric_exp = generate_correlated_variables(corr, mu_pre, std_pre, mu_exp, std_exp, n)

    is_treatment = np.concatenate((np.zeros(n//2, dtype = int), np.ones(n//2, dtype = int)), axis=None)
    np.random.shuffle(is_treatment)

    data_no_treatment = pd.DataFrame({'metric_pre': metric_pre,
                         'metric_exp': metric_exp,
                         'is_treatment': is_treatment})
    
    # simple for AA
    
    control_one = data_no_treatment[data_no_treatment.is_treatment == 0]
    control_two = data_no_treatment[data_no_treatment.is_treatment == 1]
        
    _, pvalue_aa = stats.ttest_ind(control_one['metric_exp'], control_two['metric_exp'])
    simple_first_type_errors.append(pvalue_aa < alpha)
    
    # simple for AB
    
    data_with_treatment = copy.deepcopy(data_no_treatment)
    data_with_treatment.loc[data_with_treatment.is_treatment == 1, 'metric_exp'] += treatment_effect
    
    control = data_with_treatment[data_with_treatment.is_treatment == 0]
    pilot = data_with_treatment[data_with_treatment.is_treatment == 1]
    
    _, pvalue_ab = stats.ttest_ind(control['metric_exp'], pilot['metric_exp'])
    simple_second_type_errors.append(pvalue_ab >= alpha)
    
    simple_deltas.append(pilot['metric_exp'].mean() - control['metric_exp'].mean())
    
    # CUPED for AA
    
    cov_X_Y = np.cov(data_no_treatment['metric_pre'], 
                     data_no_treatment['metric_exp'],
                     ddof=1)[0, 1]

    var_X = np.var(data_no_treatment['metric_pre'], ddof=1)

    theta = cov_X_Y / var_X
    
    data_no_treatment['metric_exp_cuped'] = data_no_treatment['metric_exp'] - \
                                        (theta * (data_no_treatment['metric_pre'] - data_no_treatment['metric_pre'].mean()))
    
    control_one = data_no_treatment[data_no_treatment.is_treatment == 0]

    control_two = data_no_treatment[data_no_treatment.is_treatment == 1]
    
    _, pvalue_aa = stats.ttest_ind(control_one['metric_exp_cuped'], control_two['metric_exp_cuped'])
    cuped_first_type_errors.append(pvalue_aa < alpha)
    
    # CUPED for AB
    
    cov_X_Y = np.cov(data_with_treatment['metric_pre'], 
                     data_with_treatment['metric_exp'],
                     ddof=1)[0, 1]

    var_X = np.var(data_with_treatment['metric_pre'], ddof=1)

    theta = cov_X_Y / var_X
    
    data_with_treatment['metric_exp_cuped'] = data_with_treatment['metric_exp'] - \
                                        (theta * (data_with_treatment['metric_pre'] - data_with_treatment['metric_pre'].mean()))
    
    control = data_with_treatment[data_with_treatment.is_treatment == 0]

    pilot = data_with_treatment[data_with_treatment.is_treatment == 1]
      
    _, pvalue_ab = stats.ttest_ind(control['metric_exp_cuped'], pilot['metric_exp_cuped'])
    cuped_second_type_errors.append(pvalue_ab >= alpha)
    
    cuped_deltas.append(pilot['metric_exp_cuped'].mean() - control['metric_exp_cuped'].mean())

prop_simple_first_type_errors = np.mean(simple_first_type_errors)
prop_simple_second_type_errors = np.mean(simple_second_type_errors)
print(f'prop_simple_first_type_errors = {prop_simple_first_type_errors:0.3f}')
print(f'prop_simple_second_type_errors = {prop_simple_second_type_errors:0.3f}')

prop_cuped_first_type_errors = np.mean(cuped_first_type_errors)
prop_cuped_second_type_errors = np.mean(cuped_second_type_errors)
print(f'prop_cuped_first_type_errors = {prop_cuped_first_type_errors:0.3f}')
print(f'prop_cuped_second_type_errors = {prop_cuped_second_type_errors:0.3f}')

print(f'Произошло снижение ошибки II рода в {round(prop_simple_second_type_errors / prop_cuped_second_type_errors, 2)} раз(а)')

In [ ]:
sns.displot({'Simple': simple_deltas, 'CUPED': cuped_deltas}, kind='kde')
plt.title('Распределение разницы средних')
plt.ylabel('Плотность')

simple_var_deltas = np.array(simple_deltas).var(ddof = 1)
cuped_var_deltas = np.array(cuped_deltas).var(ddof = 1)

print(f'Дисперсия разницы средних при обычном АВ-тесте: {simple_var_deltas}')
print(f'Дисперсия разницы средних при АВ-тесте с использованием CUPED-преобразования: {cuped_var_deltas}')

print(f'Произошло снижение дисперсии разницы средних в {round(simple_var_deltas / cuped_var_deltas, 2)} раз(а)')

**CUPED снизил дисперсию разницы средних (и соответственно ошибку II рода) больше, чем стратификация**

# CUMPED

Добавляем в модель не одну, а несколько ковариант:

## Данные

In [ ]:
def generate_several_correlated_variables(mu_x, std_x, params, n):
    '''
    Генерирует несколько случайных величин с заранее заданной корреляцией.
    Предварителньо задаются параментры величин:
    mu_x, std_x - математическое ожидание и стандартное отклонение СВ X
    params - cписок кортежей (mu, std, rho) с математическими ожиданиями и стандартными
    отклонениями других СВ, заданной корреляцией Пирсона между СВ Х и другими СВ
    n - число наблюдений
    '''
    
    Z_1 = stats.norm().rvs(n)
    Z_2 = stats.norm().rvs(n)
    
    X = std_x * Z_1 + mu_x
    
    result = pd.DataFrame({'metric': X})
    
    for n, (mu_other, std_other, rho) in enumerate(params):
        other = std_other * (rho * Z_1 + (1 - rho ** 2)**0.5 * Z_2) + mu_other
        result[f'X{n}'] = other

    return result

In [ ]:
n = 2*1100

treatment_effect = 50

mu_metr = 2500
std_metr = 800
params = [(2500, 800, 0.75), (2250, 500, 0.75), (2800, 900, 0.75)]

data = generate_several_correlated_variables(mu_metr, std_metr, params, n)

is_treatment = np.concatenate((np.zeros(n//2, dtype = int), np.ones(n//2, dtype = int)), axis=None)
np.random.shuffle(is_treatment)
data['is_treatment'] = is_treatment
data.loc[data.is_treatment == 1, 'metric'] += treatment_effect

data.head()

In [ ]:
print('Среднее значение метрики:')
print(f"В контрольной группе: {round(data[data.is_treatment == 0]['metric'].mean(), 2)}")
print(f"В тестовой группе: {round(data[data.is_treatment == 1]['metric'].mean(), 2)}")
print()
print('Стандартное отклонение метрики:')
print(f"В контрольной группе: {round(data[data.is_treatment == 0]['metric'].std(ddof = 1), 2)}")
print(f"В тестовой группе: {round(data[data.is_treatment == 1]['metric'].std(ddof = 1), 2)}")

## Парная регрессия

In [ ]:
model = smf.ols('metric ~ is_treatment', data = data).fit()
model.summary().tables[1]

In [ ]:
print(f'{model.pvalues.is_treatment:0.15f}')

## Множественная регрессия

In [ ]:
model = smf.ols('metric ~ X0 + X1 + X2 + is_treatment', data = data).fit()
model.summary().tables[1]

In [ ]:
print(f'{model.pvalues.is_treatment:0.15f}')

## CUMPED

In [ ]:
model = smf.ols('metric ~ X0 + X1 + X2 + is_treatment', data = data).fit()
model.summary().tables[1]

In [ ]:
data['metric_cumped'] = data['metric'] - (model.params[0] + \
                                            model.params[1] * data['X0'] + \
                                            model.params[2] * data['X1'] + \
                                            model.params[3] * data['X2']) + data['metric'].mean()
data.head()

In [ ]:
_, pvalue = stats.ttest_ind(data[data['is_treatment'] == 1].metric_cumped, 
                            data[data['is_treatment'] == 0].metric_cumped)
print(f'{pvalue:0.10f}')

In [ ]:
print(f"Дисперсия исходной метрики: {data['metric'].var(ddof = 1)}")
print(f"Дисперсия метрики после cumped-преобразования: {data['metric_cumped'].var(ddof = 1)}")
print(f"Дисперсия метрики снизилась в {round(data['metric'].var(ddof = 1) / data['metric_cumped'].var(ddof = 1), 2)} раз(а)")

## Визуализация

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

ax_1 = axs[0]
ax_2 = axs[1]

sns.kdeplot(x = data[data.is_treatment == 0]['metric'], 
            data = data, label = 'Initial', fill = True, alpha = 0.1, color = 'red', ax = ax_1)
sns.kdeplot(x = data[data.is_treatment == 0]['metric_cumped'], 
            data = data, label = 'CUMPED transformed', fill = True, alpha = 0.1, color = 'blue', ax = ax_1)
ax_1.set_title('Without treatment')
ax_1.legend()

sns.kdeplot(x = data[data.is_treatment == 1]['metric'],
            data = data, label = 'Initial', fill = True, alpha = 0.1, color = 'red', ax = ax_2)
sns.kdeplot(x = data[data.is_treatment == 1]['metric_cumped'], 
            data = data, label = 'CUMPED transformed', fill = True, alpha = 0.1, color = 'blue', ax = ax_2)
ax_2.set_title('With treatment')
ax_2.legend()

fig, axs = plt.subplots(1, 1, figsize=(16, 6))
sns.kdeplot(x = 'metric', 
            data = data, label = 'Initial', fill = True, alpha = 0.1, color = 'red', ax = axs)
sns.kdeplot(x = 'metric_cumped', 
            data = data, label = 'CUMPED transformed', fill = True, alpha = 0.1, color = 'blue', ax = axs)
axs.set_title('All users')
axs.legend()
plt.show();

## Ошибки I и II рода

In [ ]:
n = 1100*2

alpha = 0.05

treatment_effect = 100

mu_metr = 2500
std_metr = 800
params = [(2500, 800, 0.8), (2250, 500, 0.8), (2800, 900, 0.8)]

simple_first_type_errors = []
simple_second_type_errors = []

cumped_first_type_errors = []
cumped_second_type_errors = []

simple_deltas = []
cumped_deltas = []

for _ in tqdm.tqdm(list(range(10000))):
    
    data = generate_several_correlated_variables(mu_metr, std_metr, params, n)

    is_treatment = np.concatenate((np.zeros(n//2, dtype = int), np.ones(n//2, dtype = int)), axis=None)
    np.random.shuffle(is_treatment)
    data['is_treatment'] = is_treatment

    data_no_treatment = copy.deepcopy(data)
    
    # simple for AA
    
    control_one = data_no_treatment[data_no_treatment.is_treatment == 0]
    control_two = data_no_treatment[data_no_treatment.is_treatment == 1]
        
    _, pvalue_aa = stats.ttest_ind(control_one['metric'], control_two['metric'])
    simple_first_type_errors.append(pvalue_aa < alpha)
    
    # simple for AB
    
    data_with_treatment = copy.deepcopy(data)
    data_with_treatment.loc[data_with_treatment.is_treatment == 1, 'metric'] += treatment_effect
    
    control = data_with_treatment[data_with_treatment.is_treatment == 0]
    pilot = data_with_treatment[data_with_treatment.is_treatment == 1]
    
    _, pvalue_ab = stats.ttest_ind(control['metric'], pilot['metric'])
    simple_second_type_errors.append(pvalue_ab >= alpha)
    
    simple_deltas.append(pilot['metric'].mean() - control['metric'].mean())
    
    # CUMPED for AA
    
    model = smf.ols('metric ~ X0 + X1 + X2 + is_treatment', data = data_no_treatment).fit()
    data_no_treatment['metric_cumped'] = data_no_treatment['metric'] - (model.params[0] + \
                                              model.params[1] * (data_no_treatment['X0'] - data_no_treatment['X0'].mean()) + \
                                              model.params[2] * (data_no_treatment['X1'] - data_no_treatment['X1'].mean()) + \
                                              model.params[3] * (data_no_treatment['X2'] - data_no_treatment['X2'].mean()))
    
    control_one = data_no_treatment[data_no_treatment.is_treatment == 0]

    control_two = data_no_treatment[data_no_treatment.is_treatment == 1]
    
    _, pvalue_aa = stats.ttest_ind(control_one['metric_cumped'], control_two['metric_cumped'])
    cumped_first_type_errors.append(pvalue_aa < alpha)
    
    # CUMPED for AB
    
    model = smf.ols('metric ~ X0 + X1 + X2 + is_treatment', data = data_with_treatment).fit()
    data_with_treatment['metric_cumped'] = data_with_treatment['metric'] - (model.params[0] + \
                                              model.params[1] * (data_with_treatment['X0'] - data_with_treatment['X0'].mean()) + \
                                              model.params[2] * (data_with_treatment['X1'] - data_with_treatment['X1'].mean()) + \
                                              model.params[3] * (data_with_treatment['X2'] - data_with_treatment['X2'].mean()))
    
    control = data_with_treatment[data_with_treatment.is_treatment == 0]

    pilot = data_with_treatment[data_with_treatment.is_treatment == 1]
    
    _, pvalue_ab = stats.ttest_ind(control['metric_cumped'], pilot['metric_cumped'])
    cumped_second_type_errors.append(pvalue_ab >= alpha)
    
    cumped_deltas.append(pilot['metric_cumped'].mean() - control['metric_cumped'].mean())

prop_simple_first_type_errors = np.mean(simple_first_type_errors)
prop_simple_second_type_errors = np.mean(simple_second_type_errors)
print(f'prop_simple_first_type_errors = {prop_simple_first_type_errors:0.3f}')
print(f'prop_simple_second_type_errors = {prop_simple_second_type_errors:0.3f}')

prop_cumped_first_type_errors = np.mean(cumped_first_type_errors)
prop_cumped_second_type_errors = np.mean(cumped_second_type_errors)
print(f'prop_cumped_first_type_errors = {prop_cumped_first_type_errors:0.3f}')
print(f'prop_cumped_second_type_errors = {prop_cumped_second_type_errors:0.3f}')

print(f'Произошло снижение ошибки II рода в {round(prop_simple_second_type_errors / prop_cumped_second_type_errors, 2)} раз(а)')

In [ ]:
sns.displot({'Simple': simple_deltas, 'CUMPED': cumped_deltas}, kind='kde')
plt.title('Распределение разницы средних')
plt.ylabel('Плотность')

simple_var_deltas = np.array(simple_deltas).var(ddof = 1)
cumped_var_deltas = np.array(cumped_deltas).var(ddof = 1)

print(f'Дисперсия разницы средних при обычном АВ-тесте: {simple_var_deltas}')
print(f'Дисперсия разницы средних при АВ-тесте с использованием CUMPED-преобразования: {cumped_var_deltas}')

print(f'Произошло снижение дисперсии разницы средних в {round(simple_var_deltas / cumped_var_deltas, 2)} раз(а)')

# CUPAC

## Данные

In [ ]:
n = 2*1100

treatment_effect = 100

mu_metr = 2500
std_metr = 800
params = [(2500, 800, 0.75), (2250, 500, 0.75), (2800, 900, 0.75)]

data_train = generate_several_correlated_variables(mu_metr, std_metr, params, n)
data_train.head()

In [ ]:
n = 2*1100

treatment_effect = 100

mu_metr = 2500
std_metr = 800
params = [(2500, 800, 0.75), (2250, 500, 0.75), (2800, 900, 0.75)]

data_test = generate_several_correlated_variables(mu_metr, std_metr, params, n)

is_treatment = np.concatenate((np.zeros(n//2, dtype = int), np.ones(n//2, dtype = int)), axis=None)
np.random.shuffle(is_treatment)
data_test['is_treatment'] = is_treatment
data_test.loc[data_test.is_treatment == 1, 'metric'] += treatment_effect

data_test.head()

## Парная регрессия

In [ ]:
model = smf.ols('metric ~ is_treatment', data = data_test).fit()
model.summary().tables[1]

In [ ]:
print(f'{model.pvalues.is_treatment:0.15f}')

## Множественная регрессия

In [ ]:
model = smf.ols('metric ~ X0 + X1 + X2 + is_treatment', data = data_test).fit()
model.summary().tables[1]

In [ ]:
print(f'{model.pvalues.is_treatment:0.15f}')

## CUPAC

In [ ]:
model = RandomForestRegressor(n_estimators = 50, 
                              criterion = 'squared_error', 
                              max_depth = 7)
model.fit(X = data_train[['X0', 'X1', 'X2']], y = data_train['metric'])

In [ ]:
data_test['metric_cupac'] = data_test['metric'] - model.predict(data_test[['X0', 'X1', 'X2']]) + data_test['metric'].mean()
data_test.head()

In [ ]:
_, pvalue = stats.ttest_ind(data_test[data_test['is_treatment'] == 1].metric_cupac, 
                            data_test[data_test['is_treatment'] == 0].metric_cupac)
print(f'{pvalue:0.10f}')

In [ ]:
print(f"Дисперсия исходной метрики: {data_test['metric'].var(ddof = 1)}")
print(f"Дисперсия cupac-преобразованной метрики: {data_test['metric_cupac'].var(ddof = 1)}")
print(f"После преобразования стандартное отклонение метрики уменьшилась на {round(100 - data_test['metric_cupac'].var(ddof = 1) * 100 / data_test['metric'].var(ddof = 1), 2)}%")

## Визуализация

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 6))

ax_1 = axs[0]
ax_2 = axs[1]

sns.kdeplot(x = 'metric', 
            data = data_test[data_test.is_treatment == 0], label = 'Initial', fill = True, alpha = 0.1, color = 'red', ax = ax_1)
sns.kdeplot(x = 'metric_cupac', 
            data = data_test[data_test.is_treatment == 0], label = 'CUPAC transformed', fill = True, alpha = 0.1, color = 'blue', ax = ax_1)
ax_1.set_title('Without treatment')
ax_1.legend()

sns.kdeplot(x = 'metric',
            data = data_test[data_test.is_treatment == 1], label = 'Initial', fill = True, alpha = 0.1, color = 'red', ax = ax_2)
sns.kdeplot(x = 'metric_cupac', 
            data = data_test[data_test.is_treatment == 1], label = 'CUPAC transformed', fill = True, alpha = 0.1, color = 'blue', ax = ax_2)
ax_2.set_title('With treatment')
ax_2.legend()

fig, axs = plt.subplots(1, 1, figsize=(16, 6))
sns.kdeplot(x = 'metric', 
            data = data_test, label = 'Initial', fill = True, alpha = 0.1, color = 'red', ax = axs)
sns.kdeplot(x = 'metric_cupac', 
            data = data_test, label = 'CUPAC transformed', fill = True, alpha = 0.1, color = 'blue', ax = axs)
axs.set_title('All users')
axs.legend()
plt.show();

## Ошибки I и II рода

In [ ]:
n = 1100*2

alpha = 0.05

treatment_effect = 100

mu_metr = 2500
std_metr = 800
params = [(2500, 800, 0.8), (2250, 500, 0.8), (2800, 900, 0.8)]

simple_first_type_errors = []
simple_second_type_errors = []

cupac_first_type_errors = []
cupac_second_type_errors = []

simple_deltas = []
cupac_deltas = []

for _ in tqdm.tqdm(list(range(10000))):
    
    data_train = generate_several_correlated_variables(mu_metr, std_metr, params, n)
    
    model = RandomForestRegressor(n_estimators = 50, 
                              criterion = 'squared_error', 
                              max_depth = 7).fit(X = data_train[['X0', 'X1', 'X2']], y = data_train['metric'])
 
    data_test = generate_several_correlated_variables(mu_metr, std_metr, params, n)

    is_treatment = np.concatenate((np.zeros(n//2, dtype = int), np.ones(n//2, dtype = int)), axis=None)
    np.random.shuffle(is_treatment)
    data_test['is_treatment'] = is_treatment
    
    data_no_treatment = copy.deepcopy(data_test)
    
    # simple for AA
    
    control_one = data_no_treatment[data_no_treatment.is_treatment == 0]
    control_two = data_no_treatment[data_no_treatment.is_treatment == 1]
        
    _, pvalue_aa = stats.ttest_ind(control_one['metric'], control_two['metric'])
    simple_first_type_errors.append(pvalue_aa < alpha)
    
    # simple for AB
    
    data_with_treatment = copy.deepcopy(data_test)
    data_with_treatment.loc[data_with_treatment.is_treatment == 1, 'metric'] += treatment_effect
    
    control = data_with_treatment[data_with_treatment.is_treatment == 0]
    pilot = data_with_treatment[data_with_treatment.is_treatment == 1]
    
    _, pvalue_ab = stats.ttest_ind(control['metric'], pilot['metric'])
    simple_second_type_errors.append(pvalue_ab >= alpha)
    
    simple_deltas.append(pilot['metric'].mean() - control['metric'].mean())
    
    # CUMPED for AA
    
    data_no_treatment['metric_cupac'] = data_no_treatment['metric'] - \
                                        model.predict(data_no_treatment[['X0', 'X1', 'X2']]) + \
                                        data_no_treatment['metric'].mean()
    
    control_one = data_no_treatment[data_no_treatment.is_treatment == 0]

    control_two = data_no_treatment[data_no_treatment.is_treatment == 1]
    
    _, pvalue_aa = stats.ttest_ind(control_one['metric_cupac'], control_two['metric_cupac'])
    cupac_first_type_errors.append(pvalue_aa < alpha)
    
    # CUMPED for AB
    
    data_with_treatment['metric_cupac'] = data_with_treatment['metric'] - \
                                          model.predict(data_with_treatment[['X0', 'X1', 'X2']]) + \
                                          data_with_treatment['metric'].mean()
    
    control = data_with_treatment[data_with_treatment.is_treatment == 0]

    pilot = data_with_treatment[data_with_treatment.is_treatment == 1]
    
    _, pvalue_ab = stats.ttest_ind(control['metric_cupac'], pilot['metric_cupac'])
    cupac_second_type_errors.append(pvalue_ab >= alpha)
    
    cupac_deltas.append(pilot['metric_cupac'].mean() - control['metric_cupac'].mean())

prop_simple_first_type_errors = np.mean(simple_first_type_errors)
prop_simple_second_type_errors = np.mean(simple_second_type_errors)
print(f'prop_simple_first_type_errors = {prop_simple_first_type_errors:0.3f}')
print(f'prop_simple_second_type_errors = {prop_simple_second_type_errors:0.3f}')

prop_cupac_first_type_errors = np.mean(cupac_first_type_errors)
prop_cupac_second_type_errors = np.mean(cupac_second_type_errors)
print(f'prop_cupac_first_type_errors = {prop_cupac_first_type_errors:0.3f}')
print(f'prop_cupac_second_type_errors = {prop_cupac_second_type_errors:0.3f}')

print(f'Произошло снижение ошибки II рода в {round(prop_simple_second_type_errors / prop_cupac_second_type_errors, 2)} раз(а)')

In [ ]:
sns.displot({'Simple': simple_deltas, 'CUPAC': cupac_deltas}, kind='kde')
plt.title('Распределение разницы средних')
plt.ylabel('Плотность')

simple_var_deltas = np.array(simple_deltas).var(ddof = 1)
cupac_var_deltas = np.array(cupac_deltas).var(ddof = 1)

print(f'Дисперсия разницы средних при обычном АВ-тесте: {simple_var_deltas}')
print(f'Дисперсия разницы средних при АВ-тесте с использованием CUPAC-преобразования: {cupac_var_deltas}')

print(f'Произошло снижение дисперсии разницы средних в {round(simple_var_deltas / cupac_var_deltas, 2)} раз(а)')

Материалы по теме исследования:

* https://youtu.be/KvIJ8FCJzr4
* https://youtu.be/4_J5pvdG35U
* https://youtu.be/saeAPdTTfTM
* https://youtu.be/Qrz04qUMgVc
* https://youtu.be/rhpzdPRIxBk
* https://github.com/bdemeshev/cuped_statistician_viewpoint/blob/main/cuped_stat_viewpoint.pdf
* http://quantile.ru/06/06-JW.pdf
* http://habr.com/ru/companies/X5Tech/articles/768008